# Word Clustering by Root

Given a corpus of words extracted from texts, we may wish to cluster the words by their root stem. In order to do this, we must first remove the root stem.

I am using a corpus of words scraped from [childrens stories](latvian/books/pasakas.net.ipynb)

In [14]:
import json
import numpy as np
from pprint import pprint
import re

with open("library.json") as f:
    text = ' '.join(
        book["text"]
        for title, book in json.loads(f.read()).items()
    )
    words = list(map(str.lower, re.findall(r'[^\W\d]+', text)))

print(words)

['reiz', 'viena', 'meita', 'gājusi', 'pa', 'ceļu', 'pa', 'priekšu', 'gājis', 'vecs', 'vecis', 'kas', 'tikko', 'varējis', 'pavazāt', 'kājas', 'vecais', 'tēv', 'sēdi', 'man', 'kukurī', 'teikusi', 'meita', 'es', 'tevi', 'panesīšu', 'ko', 'nu', 'meit', 'teicis', 'vecis', 'gan', 'jau', 'es', 'tāpat', 'aizvilkšos', 'nāc', 'vien', 'nāc', 'teikusi', 'meita', 'es', 'esmu', 'jauna', 'un', 'stipra', 'tevi', 'panest', 'varu', 'nu', 'vecis', 'ar', 'nācis', 'meita', 'to', 'paņēmusi', 'uz', 'muguras', 'un', 'nesusi', 'bet', 'vecis', 'palicis', 'arvienu', 'smagāks', 'un', 'smagāks', 'un', 'beidzot', 'to', 'meita', 'vairs', 'tikko', 'varējusi', 'panest', 'tu', 'paliec', 'arvienu', 'smagāks', 'viņa', 'teikusi', 'tā', 'jau', 'varot', 'ar', 'būt', 'noteicis', 'vecis', 'un', 'rausies', 'zemē', 'nu', 'meita', 'prasījusi', 'kas', 'tu', 'tāds', 'esi', 'es', 'esmu', 'tas', 'visulabākais', 'atbildējis', 'vecis', 'kā', 'visulabākais', 'prasījusi', 'meita', 'nu', 'tad', 'jau', 'tu', 'esi', 'pats', 'dievs', 'jā', 

# Infinite parameter model

This was a rough hack that I am formalizing so that improvements can more easily be made.

Let us assume that a word is made of a prefix, stem and suffix. Each can vary in length, but the obvious order is constrained. Using our corpus of text, we can easily compute the probability of any prefix, stem and suffix. so given cut locations $ c_{prefix}, c_{suffix} $, we can compute the probability of the associated prefix, stem and suffix word the *i*th word (respectively $ P_i, T_i, S_i $).

$$\begin{align}
Pr(P_i, T_i, S_i ; \vec{c}_i)
\end{align}$$

We wish to optimized the log-likelihood of the parameters $ \{ \vec{c}_i \}_i $. Fortunately in this kind of stupid model, there is no limit on the parameters because we do it per word, so the log-likelihood factors easily and we can simply optimize per word.

$$\begin{align}
\vec{c} &= \text{arg max}_{\vec{c}} \sum_i \log Pr(P_i, T_i, S_i ; \vec{c}_i) \\
\vec{c}_i &= \text{arg max}_{\vec{c}_i} \log Pr(P_i, T_i, S_i ; \vec{c}_i)
\end{align}$$

Clearly, a reduced parameter model could have benefits, but this was easy to program for now.

In [86]:
def loglikelihood(word, words, p_cut, s_cut):
    Pr = np.sum([
        np.array([
            word[:p_cut] == w[:p_cut],
            word[p_cut:s_cut] in w,
            word[s_cut:] == w[s_cut:]
        ])
        for w in words
    ], axis=0)/len(words)
    L = np.array([p_cut, s_cut - p_cut, len(word) - s_cut])
    return np.log(Pr).dot(L), (word[:p_cut], word[p_cut:s_cut], word[s_cut:])

def max_ll(word, words):
    l_min, conf = np.inf, None
    p_min, s_min = None, None
    for p_cut in range(len(word)):
        for s_cut in range(p_cut, len(word)):
            ll, cuts = loglikelihood(word, words, p_cut, s_cut)
            if info > l_min:
                l_min, conf, p_min, s_min = ll, cuts, p_cut, s_cut
    return conf, l_min, p_min, s_min

print(min_info('mācītājs', words))
print(min_info('nonesīšot', words))

(('mācīt', 'āj', 's'), 49.39630998974996, 5, 7)
(('no', 'nes', 'īšot'), 58.99538681815128, 2, 5)


Pretty cool! It actually seems to work fairly well. Pretty good for a rough hack.